In [1]:
import re
import string
import random
import markovify

import numpy as np
import tensorflow as tf

from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import sequence

## Форматирование текста

In [2]:
def get_text(filepath):
    with open(filepath, 'r') as f:
        raw = f.read()
    raw = raw.lower()
    raw = re.sub(r'[^а-яА-Я.!?…\n]+', ' ', raw)
    raw = re.sub(r'\n+', '\n', raw)
    return re.sub(r' +', ' ', raw)

def get_vocab(text):
    res = []
    rev = {}
    for c in text:
        if c not in res:
            rev[c] = len(res)
            res.append(c)
    return rev, res

def encode(s, remap):
    return [remap[c] for c in s]

def div(spltd, max_len):
    res = []
    for spl in spltd:
        spl += '.'
        res += [spl[i:i+max_len] for i in range(0, len(spl), max_len)]
    return res

maxlen = 80
bsize = 128

In [3]:
zar = get_text('zaratustra.txt')
print('Text length:', len(zar))
voc, revoc = get_vocab(zar)
count = len(revoc)
print('Vocab power:', count)

Text length: 460172
Vocab power: 38


In [4]:
# I really tried but it all only got bad things
# splitted = re.split(r'[\?\!\.…]+', zar)
# splitted = div(splitted, maxlen)
# splitted = filter(lambda x: len(x) != 0, zar.split())
# splitted = [encode(s, voc) + [voc[' ']] for s in splitted]
# splitted[:5]
# maxlen = max(map(len, splitted))

In [5]:
# filled = sequence.pad_sequences(splitted, maxlen=maxlen+1, value=voc[' '])
# filled = np.array(encode(zar, voc))
filled = []
for i in range(0, len(zar) - maxlen - 1, 1):
    filled.append(encode(zar[i:i+maxlen+1], voc))
filled = sequence.pad_sequences(filled, maxlen=maxlen+1, value=voc[' '])
filled, filled.shape

(array([[ 0,  1,  2, ...,  8,  4,  6],
        [ 1,  2,  3, ...,  4,  6,  7],
        [ 2,  3,  4, ...,  6,  7, 15],
        ...,
        [17, 14,  6, ...,  6, 20, 15],
        [14,  6,  7, ..., 20, 15,  9],
        [ 6,  7,  8, ..., 15,  9, 26]], dtype=int32),
 (460091, 81))

In [6]:
char_dataset = tf.data.Dataset.from_tensor_slices(filled.reshape((-1,)))
sequences = char_dataset.batch(maxlen+1, drop_remainder=True)

def splt(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(splt)

dataset = dataset.shuffle(1000).batch(bsize, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 80), (128, 80)), types: (tf.int32, tf.int32)>

## LSTM section

In [7]:
# TODO more logic for arch
def build_model(vocab_size, batch_size):
    model = models.Sequential([
        layers.Embedding(vocab_size, 64, batch_input_shape=[batch_size, None]),
        layers.LSTM(256, return_sequences=True, stateful=True),
        layers.Dropout(0.2),
        layers.Dense(vocab_size, activation='softmax')
    ])
    return model

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='chpts_one/{epoch}', save_weights_only=True)

In [8]:
def generate_text(model, start_string, length, randomness):
    start = encode(start_string, voc)
    start = tf.expand_dims(start, 0)
    
    result = []

    model.reset_states()
    i = 0
    while (True):
        predictions = model(start)
        predictions = tf.squeeze(predictions, 0)
        
        predictions *= randomness
        
        predicted_id = tf.math.argmax(predictions, axis=1)[-1]
        predicted_random_shit = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        # print(encode(tf.math.argmax(predictions, axis=1), revoc))
        # print(tf.random.categorical(predictions, num_samples=1)[-1,0]) # [-1,0].numpy()

        
        start = tf.expand_dims([predicted_random_shit], 0)

        result.append(revoc[predicted_random_shit])
        # result += encode(predicted_ids, revoc)
        
        # print(len(result), result[-1])
        i += 1
        if (i >= length and result[-1] in ['.', '!', '?', '…']) or (i >= length * 1.5):
            break

    return (start_string + ''.join(result))

In [35]:
model = build_model(len(voc), bsize)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (128, None, 64)           2432      
_________________________________________________________________
lstm_5 (LSTM)                (128, None, 256)          328704    
_________________________________________________________________
dropout_5 (Dropout)          (128, None, 256)          0         
_________________________________________________________________
dense_5 (Dense)              (128, None, 38)           9766      
Total params: 340,902
Trainable params: 340,902
Non-trainable params: 0
_________________________________________________________________


In [ ]:
i, more = 0, 50
model.fit(dataset, initial_epoch=i, epochs=i+more, callbacks=[checkpoint])

In [9]:
sbatch_model = build_model(len(voc), 1)
sbatch_model.load_weights(tf.train.latest_checkpoint('chpts'))
sbatch_model.build(tf.TensorShape([1, None]))

In [10]:
print(generate_text(sbatch_model, 'заратустра сказал', 2000, 30))

заратустра сказал бы своего вернулся еще с полночным и еще оставался о высшие люди как я услыхал хочет вечности мои свой воскликнул он с полночным и любви подобно моей  хочет моей вещей мои сострадание хочет мне и потом его в себя самый пещере своим сердце своем и полночь теперь себя самого всех сегодня о заратустра в сердце своем и простил себе себя на себя самих себя с полночным и полночь меня в себя самого своего сказал он в сердце своем и цели с поыснился с восклезном делало ли вы не ! 
как еще слишком много своего и от всех вещий друг так велики стали вы высшие люди стал по высшие люди на которой уни в последний человек в полночь мое сострадание не заратустра и лицо свой полдень и хочет домой приближается о высшие люди мне не отвечал самый безобразный человек но еще совершенно пространовился ? 
 
большом у любовь к нему и сказал заратустра и слово своей  головой справидали о как между вечер своего долго более простите в лицо свой из своего и сам самого своего вернул он совершенно 

# Markov chain

In [12]:
corpus = list(map(lambda x: list(x + '.'), re.split(r'[\?\!\.…]+', zar)))

In [13]:
chain = markovify.Chain(corpus=corpus, state_size=5)
chain.compile()
print(' '.join([''.join(chain.walk()) for _ in range(100)]))


самым что и если б не бойкие души. 
никогда прежде чем моим духе ваша волосы.  ну что лгут .  как тешила ли похожим назначения которые пристыженные подумал он с тобой тени его ограбил. 
даже тюрьму. 
но род род при этом оно глаза.  
пусть вашу любовь и наравнение горит добродетель.  
последние своей добродетели.  двигать это.  вознести их.  куча беремени низкий из любви так охотишься тем кто сумрака много у них .  о том что однажды от тебя.  но том его смеялась я смотрите числу паразить добре и жалкие мое 
 хотел я себе братья мои.  какое нибудь более жаждут сидит наших людям .  поступать вас не среди подобных баловней и от чрезмерно бороду служим также ведомые света между двух которыми ядовитыми ходил к нему недостаточно для чего смирению когда присоедино. 
пиявки лежала их мачтах почествовалась сердитесь от мир сзади них негодующий. 
ты называли бы мне самого вы какой также ушел месяца в придется к пучины глупостью 
 о заратустра танцор ветер душ называет . 
все и для отдохновение е